In [1]:
# Import necessary library files
import numpy as np
import matplotlib.pyplot as plt
import PIL
from PIL import Image
import os
import glob
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model 
from tensorflow.keras.layers import Input, Activation, GlobalAveragePooling2D, Dense, Lambda
from tensorflow.keras import backend as K
tf.__version__

'2.4.0-rc0'

In [2]:
# Set common params
im_height = 200
im_width = 200
num_channels = 3
num_p = 20
num_im = 100

In [3]:
def random_element(exclude, range_):
    return random.choice(list(set([x for x in range(0, range_)]) - set([exclude])))

def generate_pairs(vid):
    
#     y1, y2, y3, y4
#     s1, s2, s3, s4
#     y1, yr 1
#     y1, sr 0
#     y2, yr 1
#     y2, sr 0
#     y3, yr
#     y3, sr
#     y4, yr 1
#     y4, sr, 0

#     2,4
#     im[0][2] -> 0th person, 2nd image
#     im[1][1] -> 

#     75*20*100*2
    images = np.zeros((num_p, num_im, im_height, im_width, num_channels))
    path = 'dataset/jpg_Extracted_PIDS/'+str(vid)+'/'
    folders = ([name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))])
    folders.sort() # People
    folders = folders[1:1+num_p]
    for i,folder in enumerate(folders):
        temp_path = path+folder+'/'
        files = glob.glob(temp_path+'*', recursive=True) # Image
        files = random.choices(files, k=num_im)
        for j,filename in enumerate(files):
            with Image.open(filename) as im:
                im = im.resize((im_height, im_width))
                images[i,j] = np.array(im)
                
#     plt.imshow(images[1,0].astype("uint8"))
    X = np.zeros((num_p*num_im*2, 2, im_height, im_width, num_channels)) # (16 X 2 X 200 X 200 X 3)
    Y = np.zeros((num_p*num_im*2))
    v = 0
    for i in range(num_p):
        for j in range(num_im):
            p1 = np.concatenate(([images[i][j]],[images[i][random_element(j,num_im)]]))
            p2 = np.concatenate(([images[i][j]],[images[random_element(i,num_p)][random_element(j,num_im)]]))
            X[v] = p1
            Y[v] = 1.0
            v+=1
            X[v] = p2
            Y[v] = 0.0
            v+=1
    
    return X,Y

In [4]:
X, Y = generate_pairs('12-11-2019-4-1')
print(X.shape, Y.shape)

(4000, 2, 200, 200, 3) (4000,)


In [5]:
path = 'dataset/jpg_Extracted_PIDS/'
folders = ([name for name in os.listdir(path) if os.path.isdir(os.path.join(path, name))])
# print(folders)
print("Number of videos:", len(folders))

Number of videos: 75


In [6]:
# Training with a generator
def data_generator(batch_size, eph):
    for _ in range(eph):
#         steps = 300000//batch_size
        # our steps will be 3000
        for f in folders:
            X_batch, Y_batch = generate_pairs(f) # 4000
            for i in  range(0, num_p*num_im*2, batch_size):
                yield [X_batch[i:i+batch_size, 0], X_batch[i:i+batch_size, 1]], Y_batch[i:i+batch_size]

In [7]:
base_model = tf.keras.applications.InceptionV3(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(im_height, im_width, num_channels),
    include_top=False,
)
# Freeze the base_model
base_model.trainable = False

In [8]:
def l1_distance(inputs):
    input1, input2 = inputs
    output = K.abs(input1 - input2)
    return output

def l1_distance_output_shape(shapes):
    shape1, shape2 = shapes
    assert shape1 == shape2
    return (1,)

In [9]:
def siamese_model():
    input1 = Input(shape=(im_height, im_width, num_channels,))
    input2 = Input(shape=(im_height, im_width, num_channels,))
    processed_input1 = tf.keras.applications.inception_v3.preprocess_input(input1)
    processed_input2 = tf.keras.applications.inception_v3.preprocess_input(input2)
    feature_vec1 = base_model(processed_input1)
    feature_vec2 = base_model(processed_input2)
    pool1 = GlobalAveragePooling2D()(feature_vec1) # 6x6x2048 -> 2048
    pool2 = GlobalAveragePooling2D()(feature_vec2)
    distance = Lambda(l1_distance, output_shape=l1_distance_output_shape)([pool1, pool2])
#     pool = GlobalAveragePooling2D()(distance)
    d1 = Dense(256)(distance)
    d2 = Dense(1)(d1)
    output = Activation('sigmoid')(d2)
    return Model(inputs=[input1, input2], outputs=output)

In [10]:
model = siamese_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
tf.math.truediv (TFOpLambda)    (None, 200, 200, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
tf.math.truediv_1 (TFOpLambda)  (None, 200, 200, 3)  0           input_3[0][0]                    
______________________________________________________________________________________________

In [ ]:
bs = 100
eph = 5
model.fit(data_generator(bs, eph), batch_size=bs, epochs=eph, steps_per_epoch=3000)
# model.fit([X[:,0], X[:,1]], Y, batch_size=bs, epochs=eph)

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2022-02-25 12:51:04.955307: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-02-25 12:51:04.955514: W tensorflow/core/platform/profile_utils/cpu_utils.cc:126] Failed to get CPU frequency: 0 Hz


   6/3000 [..............................] - ETA: 19:12:38 - loss: 2.5792 - binary_accuracy: 0.4918

In [11]:
base_model.trainable = True
bs = 10
eph = 5
model.fit(data_generator(bs, eph), epochs=eph, steps_per_epoch=3000)
# model.fit([X[:,0], X[:,1]], Y, epochs=eph, batch_size=bs)

Epoch 1/5
400/400 [==============================] - 176s 437ms/step - loss: 0.5913 - binary_accuracy: 0.6780
Epoch 2/5
400/400 [==============================] - 191s 477ms/step - loss: 0.5821 - binary_accuracy: 0.6865
Epoch 3/5
400/400 [==============================] - 198s 495ms/step - loss: 0.5750 - binary_accuracy: 0.6945
Epoch 4/5
400/400 [==============================] - 216s 540ms/step - loss: 0.5757 - binary_accuracy: 0.6960
Epoch 5/5
400/400 [==============================] - 203s 506ms/step - loss: 0.5653 - binary_accuracy: 0.7035


In [12]:
model.save('siamese_1vid.h5')

References:
    https://github.com/aup8497/Person-Re-identification-using-Siamese-networks/blob/master/Code/siamese_model.py

In [2]:
tf.config.list_physical_devices('GPU')

[]

In [4]:
### Aman's code to enable the GPU
from tensorflow.python.compiler.mlcompute import mlcompute
tf.compat.v1.disable_eager_execution()
mlcompute.set_mlc_device(device_name='gpu')
print("is_apple_mlc_enabled %s" % mlcompute.is_apple_mlc_enabled())
print("is_tf_compiled_with_apple_mlc %s" % mlcompute.is_tf_compiled_with_apple_mlc())
print(f"eagerly? {tf.executing_eagerly()}")
print(tf.config.list_logical_devices())

is_apple_mlc_enabled True
is_tf_compiled_with_apple_mlc True
eagerly? False
[LogicalDevice(name='/device:CPU:0', device_type='CPU')]
